## WHICH DND PATTERNS SERVE AS MOLECULAR CLOCKS? - PART 2/2

Instead of creating an algorithm that uses a local initializing variable (in our case the kmers in the first DNA string), we will try our hand at Randomized Algorithms, starting with a RandomMotifSearch algorithm that will randomly select a kmer from each DNA string to initialize the motif matrix. <br>

This process will most likey create a poor motif matrix set, however, running this algorithm many times and taking the best kmer set of motifs will yield a more accurate estimation.

#### RANDOMIZED MOTIF SEARCH 1.1

Below is our pseduo-code for the RandomizeMotifSearch.
```
RandomizedMotifSearch(Dna, k, t)
    randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
    BestMotifs ← Motifs
    while forever
        Profile ← Profile(Motifs)
        Motifs ← Motifs(Profile, Dna)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
        else
            return BestMotifs
```



In [16]:
# RANDOMIZED MOTIF SEARCH
# INPUT: K and T integers and a space separted string DNA.
# OUTPUT: The "best" motif set from the intial random selection of t motifs.

def RandomizedMotifSearch_old(DNA, k, t) -> list[str]:
  motifs = randomMotifGenerator(DNA, k)
  best_motifs = motifs.copy()
  dna_list = DNA.split()
  # print(motifs)

  # while True will continue the loop indefinitely
  count = 0
  '''
  main issue was not having an independent outer For loop that did 1000 iterations
  and THEN having the interior while loop that would run indefinitel, checking for improvements on each interation
  of the outer loop

  '''
  while count <= 1000:
    count += 1
    profile = countMotifPercentPseudocounts(motifs)
    # create a new set of motifs based on the profile
    for string in dna_list:
      nextMotif = ProfileMostProbableKmer(string, k, profile)
      motifs.append(nextMotif)
    # now score Motifs vs BestMotifs, replacing bestMotifs if Motifs has a lower score
    if computeMotifScore(motifs) < computeMotifScore(best_motifs):
      best_motifs = motifs.copy()
      motifs = []

  return best_motifs

In [17]:
# url = "/content/sample_data/dataset_30307_5 (1).txt"

# with open(url, 'r') as file:
#   lines = file.readlines()
#   k = int(lines[0].split()[0])
#   t = int(lines[0].split()[1])
#   DNA = lines[1]

# answer = RandomizedMotifSearch(DNA, k, t)
# display(" ".join(answer))

In [18]:
# RANDOM MOTIF GENERATOR
# INPUT: K integer for the lengther of the motif and a space separated string DNA
# OUTPUT: A randomly selected kmer from each string in DNA to create a motif list

import random

def randomMotifGenerator(DNA, k) -> list[str]:
  motifs = []
  dna_list = DNA.split()
  for string in dna_list:
    random_index = random.randint(0, len(string)-k)
    motif = string[random_index:random_index+k]
    motifs.append(motif)
  return motifs

# HAMMING DISTANCE
# INPUT: Two strings of equal length
# OUTPUT: The hamming distance between the two strings

def HammingDistance(p: str, q:str) -> int:
  hd = 0
  for i in range(len(p)):
    if p[i] != q[i]:
      hd += 1
  return hd


In [19]:
# -- UPDATED VERSION -- 03/25/25
# ProfileMostProbableKmer -> We will find a Profile-Most probable kmer of a string
# INPUT: a string TEXT, integer k, and a 4xk profile matrix (row x column)
# OUTPUT: A Profile-most probable kmer in TEXT.

# we are basing out rows from previous examples, order is as follows:
# 0 -> A, 1 -> C, 2 -> G, 3 -> T
def ProfileMostProbableKmer(text: str, k: int, profiles) -> str:
  mostProbableKmer = text[:k]
  kmers = set()
  prob_table = {}
  # lets list our kmers
  for i in range(len(text)-k+1):
    kmer = text[i:i+k]
    kmers.add(kmer)

  # for each kmer, we want to compute its probability based on the given probabilty matrix
  for kmer in kmers:
    # setting the base probability as 1, but this will change accordingly when multiplied by the actual probability.
    prob_table[kmer] = 1

    # for each nucleotide in kmer, we check which it is and then multiply the appropriate probability based on its position in the string.
    for column in range(k):
      # UPDATE
      # using the .get() method allows us to only obtain the probability in profile correspondin to the nucleotide and position of kmer[column]
      prob_table[kmer] = (prob_table[kmer] * profiles[column].get(kmer[column],0))

  # now we look for the max probability value in our table
  max_val = max(prob_table.values())

  # now we look for the kmer that has that max value
  for kmer in prob_table:
    if max_val == 0:
      return mostProbableKmer

    if prob_table[kmer] == max_val:
      mostProbableKmer = kmer

  return mostProbableKmer


# countMotifPercent
# INPUT: A list of DNA strings
# OUTPUT: A profile matrix, showing the probability of a nucleotide being present at different indices

def countMotifPercent(motifs):
    count = {}
    columns = []
    for i in range(len(motifs[0])):
        columns.append([motif[i] for motif in motifs])
    for i in range(len(columns)):
        count[i] = {'A': columns[i].count('A')/len(columns[i]), 'C': columns[i].count('C')/len(columns[i]), 'G': columns[i].count('G')/len(columns[i]), 'T': columns[i].count('T')/len(columns[i])}

    return count


In [20]:
# CODE FROM PREVIOUS WEEK
# --- UPDATED VERSION ---
# 03/26/25

def GreedyMotifSearch(DNA: str, k: int, t: int) -> list[str]:
    dnaList = DNA.split()
    bestMotifs = [string[:k] for string in dnaList]  # Initialize with first k-mers
    bestScore = float('inf')  # Track the best score (lower is better)

    # Iterate over all possible k-mers in the first DNA string
    for i in range(len(dnaList[0]) - k + 1):
        motifs = [dnaList[0][i:i+k]]  # Start with the current k-mer from the first sequence

        # Build the profile and find motifs in the remaining sequences
        for j in range(1, t):
            profile = countMotifPercent(motifs)
            nextMotif = ProfileMostProbableKmer(dnaList[j], k, profile)
            motifs.append(nextMotif)

        # Compute the score of the current motif set
        currentScore = computeMotifScore(motifs)

        # Update bestMotifs if the current set is better
        if currentScore < bestScore:
            bestScore = currentScore
            bestMotifs = motifs.copy()

    return bestMotifs


def computeMotifScore(motifs: list[str]) -> int:
    """Compute the total number of mismatches against the consensus motif."""
    if not motifs:
        return 0

    consensus = []
    k = len(motifs[0])

    # Build the consensus string
    for i in range(k):
        column = [motif[i] for motif in motifs]
        mostCommon = max(set(column), key=column.count)
        consensus.append(mostCommon)
    consensusMotif = ''.join(consensus)

    # Calculate total mismatches
    score = 0.0
    # for motif in motifs:
    #     score += sum(1 for a, b in zip(motif, consensusMotif) if a != b)
    for motif in motifs:
      score += HammingDistance(motif,consensusMotif)

    return score

In [21]:
def countMotifPercentPseudocounts(motifs):
    count = {}
    columns = []
    for i in range(len(motifs[0])):
        columns.append([motif[i] for motif in motifs])
    for i in range(len(columns)):
        # adding a psuedocount (+1) to remove probabilities of zero
        count[i] = {'A': (columns[i].count('A'))+1/len(columns[i])+4, 'C': (columns[i].count('C'))+1/len(columns[i])+4, 'G': (columns[i].count('G'))+1/len(columns[i])+4, 'T': (columns[i].count('T'))+1/len(columns[i])+4}

    return count

In [22]:
# testing the syntax to retrieve the probabilites of each
motif = "ACCTG"
motif_percents = countMotifPercentPseudocounts(motif)
print(motif_percents)
values = motif_percents[0].values()
print(list(values))

keys = motif_percents[0].keys()
print(list(keys)[0])

{0: {'A': 5.2, 'C': 6.2, 'G': 5.2, 'T': 5.2}}
[5.2, 6.2, 5.2, 5.2]
A


In [23]:
# --- UPDATED --- 04/03/2025

def RandomizedMotifSearch(DNA, k, t) -> list[str]:
    dna_list = DNA.split()
    best_motifs = []
    best_score = float('inf')  # Initialize with worst possible score

    for _ in range(1000):  # Run 1000 independent restarts
        # 1) Generate random starting motifs
        motifs = randomMotifGenerator(DNA, k)
        current_score = computeMotifScore(motifs)

        while True:  # Iterate until no improvement
            # 2) Build profile from current motifs
            profile = countMotifPercentPseudocounts(motifs)

            # 3) Generate new motifs using the profile
            new_motifs = []
            for string in dna_list:
                nextMotif = ProfileMostProbableKmer(string, k, profile)
                new_motifs.append(nextMotif)

            # 4) Check if new motifs are better
            new_score = computeMotifScore(new_motifs)

            if new_score < current_score:
                motifs = new_motifs
                current_score = new_score
            else:
                break  # No improvement, exit loop

        # 5) Update best_motifs if this run found a better set
        if current_score < best_score:
            best_motifs = motifs.copy()
            best_score = current_score

    return best_motifs

In [37]:
DNA = "TGACGTTC TAAGAGTT GGACGAAA  CTGTTCGC"
motifs = ["TGA","GTT","GAA","TGT"]

dna_strings = DNA.split()
profile = countMotifPercentPseudocounts(motifs)

new_motifs = []
for string in dna_strings:
  nextMotif = ProfileMostProbableKmer(string, 3, profile)
  new_motifs.append(nextMotif)

print(new_motifs)
display(" ".join(new_motifs))

['TGA', 'TAA', 'GGA', 'TGT']


'TGA TAA GGA TGT'

In [24]:
DNA = "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG TAGTACCGAGACCGAAAGAAGTATACAGGCGT TAGATCAAGTTTCAGGTGCACGTCGGTGAACC AATCCACCAGCTCCACGTGCAATGTTGGCCTA"
k = 8
t = 5

print(RandomizedMotifSearch(DNA, k, t))

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']


RandomizedMotifSearch has a large beenift of being able to run longer motifs since it has a lower time complexity than the GreedyMotifSearch.

#### GIBB'S SAMPLING 1.3

Like RandomizedMotifSearch, GibbsSampler starts with randomly chosen k-mers in each of t DNA sequences, but it makes a random rather than a deterministic choice at each iteration. It uses randomly selected k-mers (Motif1, …, Motift) to come up with another (hopefully better scoring) set of k-mers. In contrast with RandomizedMotifSearch (which deterministically defines new motifs as <br><br>

Motifs(Profile(Motifs), Dna), <br><br>

GibbsSampler randomly selects an integer i between 1 and t and then randomly changes only a single k-mer Motifi.

Creating a function RandomGenerator() that takes the probabilities given and divides each by the sum of the probabilities. Since this will be a non-negative set of probabilities, which could not sum up to 1.

In [25]:
import random
# INPUT: A list of non-negative probabilites
# OUTPUT: A list of probabilites that sum to 1
def GibbsRandomGenerator(probabilities):
  # need to return probabilities that are each divided by the sum
  # WAS RUNNING TOO SLOW
  new_probabilities = probabilities.copy()
  for i in range(len(probabilities)):
    new_probabilities[i] = new_probabilities[i]/sum(probabilities)
  # new_probabilities = new_probabilities/probabilities.sum()

  return new_probabilities

In [26]:
# INPUT: A list of non-negative probabilites
# OUTPUT: An index chosen by weighted probabilites of each index
def ChooseWeightedRandom(probabilities):
  # given numbers are a set of non-zero probabilites, lets make the set equal to 1:
  weighted_set = GibbsRandomGenerator(probabilities)

  # now that we have a weighted set, we choose an index based on the probability of the index
  random_index = random.choices(range(len(weighted_set)), weights=weighted_set, k=1)
  return random_index[0]

In [27]:
probs = [0.1, 0.2, 0.3]
n = 0
while n < 10:
  print(ChooseWeightedRandom(probs))
  n += 1
print(ChooseWeightedRandom(probs))

2
1
2
1
2
1
1
1
2
2
2


"GibbsSampler uses this random number generator to select a Profile-randomly generated k-mer at each step: if the die rolls the number i, then we define the Profile-randomly generated k-mer as the i-th k-mer in Text. While the pseudocode below repeats this procedure N times, in practice GibbsSampler depends on various stopping rules that are beyond the scope of this chapter" - *Bioinformatic Algorithms*.

Below will represent our pseudocode for the Gibbs Sampler function.

```
GibbsSampler(Dna, k, t, N)
    randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
    BestMotifs ← Motifs
    for j ← 1 to N
        i ← Random(t)
        Profile ← profile matrix constructed from all strings in Motifs except for Motifi
        Motifi ← Profile-randomly generated k-mer in the i-th sequence
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs
```



In [28]:
# GIBBS SAMPLER
# INPUT: Integers k, t, and N representing the length of our k-mers, number of DNA strings and number of times to run GIBBS SAMPLER respectively.
#     followed by space separated string DNA.
# OUTPUT: Best Motifs for the selected DNA strings, but this time randomly changing one kmer selection instead of all of them at once.

def GibbsSampler(k, t, N, DNA) -> list[str]:
  # 1) Randomly select a set of motifs from each DNA substring
  motifs = randomMotifGenerator(DNA, k)
  best_motifs = motifs.copy()
  dna_list = DNA.split()

  for _ in range(N): # this loop will run for N iterations
    # 2) randomly select an index(i) in order to remove motif(i) from our collection of motifs
    random_index = random.randint(0, t-1)
    dna_substring = dna_list[random_index]

    # 3) Now compute the profile matrix using the set of motifs, excluding the randomely selected one
    # ISSUE: If the random_index = k, then motifs[k+1] will return only the first motif as it overflows
    current_motifs = motifs.copy()
    removed_motif = current_motifs.pop(random_index)
    profile = countMotifPercentPseudocounts(motifs)
    '''
    THE NESTED FOR LOOP BELOW IS CAUSING SERIOUS TIME CONCERNS 04/17/2025
    '''
    # 4) Compute the probability of each motif in the removed DNA substring i; given our computed Profile Matrix
    kmers = []
    prob_table = {}
    for i in range(len(dna_substring)-k+1):
      kmer = dna_substring[i:i+k]
      kmers.append(kmer)
      # for each kmer, we want to know the probability basd on the givien profile matrix
      for kmer in kmers:
        prob_table[kmer] = 1

        # for each nucleotide in kmer, we check which it is and multiply the appropriate probability to the overall kmer probability
        for column in range(k):
        # using the .get() method allows us to only obtain the probability in profile correspondin to the nucleotide and position of kmer[column]
          prob_table[kmer] = prob_table[kmer] * profile[column].get(kmer[column],0)

      # 5) use weighted random selection to choose which kmer to add to our motifs
      probabilities = list(prob_table.values())
      weighted_random_index = ChooseWeightedRandom(probabilities)

      # now grabbing the kmer from the weighted-randomly index selection
      new_motif = kmers[weighted_random_index]

      # reinsert new kmer from previously removed dna_substring
      # FOUND THE PROBLEM -> I popped the kmer from motifs before, but here when I attmped to splice
      # motifs, the kmer with the random index was not here, so with each iteration the number of kmers was decreasing
      # FIX: created a working copy called "current_motifs" that is updated at the end of each iteration
      # BEFORE: motifs = motifs[:random_index] + [new_motif] + motifs[random_index+1:]
      current_motifs = motifs[:random_index] + [new_motif] + motifs[random_index+1:]
      # 6) now we check if the new set of motifs have a better motif score then the current "best" set of motifs
    # updating 'motifs' to be equal to 'current_motifs'
    motifs = current_motifs.copy()
    if computeMotifScore(motifs) < computeMotifScore(best_motifs):
      best_motifs = motifs.copy()

      # 7) repeat N times

  return best_motifs





In [33]:
# k = 8
# t = 5
# N = 100
# DNA = "CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG TAGTACCGAGACCGAAAGAAGTATACAGGCGT TAGATCAAGTTTCAGGTGCACGTCGGTGAACC AATCCACCAGCTCCACGTGCAATGTTGGCCTA"

# answer = GibbsSampler2(k,t,N,DNA)
# print(answer)
# display(" ".join(answer))
# 04/17/25 : Getting an out of bounds warning for length of motifs - FIXED on 04/17/25
# 04/17/25 : Getting the correct number of BestMotifs, but the incorrect kmers for BestMotifs - ERROR

In [32]:
def computeMotifScore2(motifs: list[str]) -> int:
    """Compute the total mismatches against the consensus, using pseudocounts."""
    if not motifs:
        return 0

    k = len(motifs[0])
    consensus = []
    bases = {'A', 'C', 'G', 'T'}

    for i in range(k):
        counts = {'A': 1, 'C': 1, 'G': 1, 'T': 1}  # Pseudocounts
        for motif in motifs:
            counts[motif[i]] += 1
        consensus.append(max(counts.items(), key=lambda x: x[1])[0])

    consensusMotif = ''.join(consensus)
    return sum(HammingDistance(motif, consensusMotif) for motif in motifs)

In [30]:
from types import new_class
import random
import numpy as np

def GibbsSampler2(k, t, N, DNA) -> list[str]:
  for _ in range(20):
      dna_list = DNA.split()
      motifs = randomMotifGenerator(DNA, k)
      best_motifs = motifs.copy()
      best_score = computeMotifScore(best_motifs)

      for _ in range(N):
          # Randomly select a sequence to exclude
          random_index = random.randint(0, t-1)
          excluded_seq = dna_list[random_index]

          # Create profile from remaining motifs
          profile = countMotifPercentPseudocounts([motif for i, motif in enumerate(motifs) if i != random_index])

          # Compute all kmer probabilities for the excluded sequence GIVEN the probability matrix "profile".
          kmers = [excluded_seq[i:i+k] for i in range(len(excluded_seq)-k+1)]
          probabilities = np.zeros(len(kmers))

          # Calculate probabilities
          for i, kmer in enumerate(kmers):
              prob = 1.0
              for pos in range(k):
                  prob *= profile[pos].get(kmer[pos], 1)
              # the index in probabilities corresponds to the index in kmers
              probabilities[i] = prob

          # Normalize probabilities and select new motif from excluded set using GibbsRandomGenerator
          # normalized_probabilities = GibbsRandomGenerator(probabilities)
          selected_idx = ChooseWeightedRandom(probabilities)
          new_motif = kmers[selected_idx]


          # Update motif at the previously randomly selected index
          motifs[random_index] = new_motif

          # Update best motifs if improved
          current_score = computeMotifScore2(motifs)
          if float(current_score) < float(best_score):
              best_motifs = motifs.copy()
              best_score = current_score
          # if N % 10 == 0:  # Print every 10th iteration
          # # SCORE NOT IMPROVING!!
          #     print(f"Iteration {_}, Current Score: {current_score}, Best Score: {best_score}")

  return best_motifs

As of 04/24/25 the GibbsSampler2 is not returning the expected strings.

In [ ]:
url = "/content/sample_data/dataset_30309_11 (3).txt"

with open(url, 'r') as file:
  lines = file.readlines()
  k = int(lines[0].split()[0])
  t = int(lines[0].split()[1])
  N = int(lines[0].split()[2])
  DNA = lines[1]


In [ ]:
answer = GibbsSampler2(k,t,N,DNA)
display(" ".join(answer))